## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
# Import API key
sys.path.append("../")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Chapais,49.7834,-74.8492,55.18,63,89,9.93,CA,overcast clouds
1,1,San Juan,-31.5375,-68.5364,53.46,44,0,4.03,AR,clear sky
2,2,Wangqing,43.3218,129.7634,67.30,96,100,6.02,CN,overcast clouds
3,3,Barrow,71.2906,-156.7887,33.82,80,90,25.32,US,light rain
4,4,Tres Picos,15.8667,-93.5333,78.51,87,94,2.24,MX,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 56


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Chapais,49.7834,-74.8492,55.18,63,89,9.93,CA,overcast clouds
1,1,San Juan,-31.5375,-68.5364,53.46,44,0,4.03,AR,clear sky
3,3,Barrow,71.2906,-156.7887,33.82,80,90,25.32,US,light rain
5,5,Hermanus,-34.4187,19.2345,52.63,69,84,9.62,ZA,broken clouds
7,7,Constitucion,-35.3333,-72.4167,49.77,71,0,4.85,CL,clear sky
8,8,Tasiilaq,65.6145,-37.6368,35.76,66,85,7.27,GL,light rain
9,9,Saldanha,-33.0117,17.9442,46.45,66,100,10.22,ZA,overcast clouds
15,15,Punta Arenas,-53.1500,-70.9167,51.91,71,0,23.02,CL,clear sky
16,16,Tazovskiy,67.4667,78.7000,35.55,66,42,16.51,RU,scattered clouds
17,17,Lebu,-37.6167,-73.6500,47.05,85,2,12.93,CL,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 686
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              230 non-null    int64  
 1   City                 230 non-null    object 
 2   Lat                  230 non-null    float64
 3   Lng                  230 non-null    float64
 4   Max Temp             230 non-null    float64
 5   Humidity             230 non-null    int64  
 6   Cloudiness           230 non-null    int64  
 7   Wind Speed           230 non-null    float64
 8   Country              228 non-null    object 
 9   Current Description  230 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 19.8+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228 entries, 0 to 686
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              228 non-null    int64  
 1   City                 228 non-null    object 
 2   Lat                  228 non-null    float64
 3   Lng                  228 non-null    float64
 4   Max Temp             228 non-null    float64
 5   Humidity             228 non-null    int64  
 6   Cloudiness           228 non-null    int64  
 7   Wind Speed           228 non-null    float64
 8   Country              228 non-null    object 
 9   Current Description  228 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 19.6+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chapais,CA,55.18,overcast clouds,49.7834,-74.8492,
1,San Juan,AR,53.46,clear sky,-31.5375,-68.5364,
3,Barrow,US,33.82,light rain,71.2906,-156.7887,
5,Hermanus,ZA,52.63,broken clouds,-34.4187,19.2345,
7,Constitucion,CL,49.77,clear sky,-35.3333,-72.4167,
8,Tasiilaq,GL,35.76,light rain,65.6145,-37.6368,
9,Saldanha,ZA,46.45,overcast clouds,-33.0117,17.9442,
15,Punta Arenas,CL,51.91,clear sky,-53.1500,-70.9167,
16,Tazovskiy,RU,35.55,scattered clouds,67.4667,78.7000,
17,Lebu,CL,47.05,clear sky,-37.6167,-73.6500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ....Skipping ")
        
       
        

 # modify it
hotel_df.head()

Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not found ....Skipping 
Hotel not 

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chapais,CA,55.18,overcast clouds,49.7834,-74.8492,Hôtel Opémiska
1,San Juan,AR,53.46,clear sky,-31.5375,-68.5364,Hotel Provincial
3,Barrow,US,33.82,light rain,71.2906,-156.7887,King Eider Inn
5,Hermanus,ZA,52.63,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Constitucion,CL,49.77,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df["Hotel Name"]==""]
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
clean_hotel_df.loc[(clean_hotel_df["Hotel Name"]=="")]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chapais,CA,55.18,overcast clouds,49.7834,-74.8492,Hôtel Opémiska
1,San Juan,AR,53.46,clear sky,-31.5375,-68.5364,Hotel Provincial
3,Barrow,US,33.82,light rain,71.2906,-156.7887,King Eider Inn
5,Hermanus,ZA,52.63,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Constitucion,CL,49.77,clear sky,-35.3333,-72.4167,Hotel Boutique Las Azucenas
...,...,...,...,...,...,...,...
668,Burnie,AU,49.06,broken clouds,-41.0667,145.9167,Beachfront Voyager Motor Inn
673,San Luis,AR,48.81,few clouds,-33.2950,-66.3356,Huarpes Hotel
677,Eina,NO,45.91,overcast clouds,60.6295,10.5993,Skogstad Leirsted
679,Juneau,US,51.80,moderate rain,58.3019,-134.4197,"Baranof Downtown, Bw Signature Collection"


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
WeatherPy_vacation_map.png